In [1]:
# !pip install openpyxl

In [20]:
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

df = pd.read_csv("/data/Augmented_data.csv")
df["Surname"] = df["Surname"].progress_apply(lambda x: str(x).capitalize())
df.head()

100%|██████████| 332397/332397 [00:00<00:00, 941663.38it/s]


,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned,date
0,15658241,Olmstead,140.686089,Germany,Female,81,48,438091.980941,2,1,0,1.0,0,0,2,GOLD,2194.0,1-2-2023
1,15627162,Blesing,614.692750,Germany,Male,27,6,155245.230616,1,1,0,1.0,0,0,1,SILVER,675.0,1-3-2020
2,15986692,Leiva,665.305005,Germany,Male,35,7,157859.682813,2,1,1,1.0,0,0,5,SILVER,758.0,1-10-2020
3,15443069,Jamieson,541.210330,France,Male,36,7,0.000000,2,0,1,0.0,1,0,5,GOLD,1365.0,1-10-2020
4,15611786,Tsui,621.240000,Spain,Female,69,9,0.000000,1,0,1,0.0,0,0,5,PLATINUM,401.0,1-1-2020


In [21]:
cus_st = {}
for cus, st in zip(df["CustomerId"],df["Geography"]):
    if cus not in cus_st:
        cus_st[cus] = [st]
    else:
        if st not in cus_st[cus]:
            cus_st[cus].append(st)

In [22]:
df["Geography"] =  df["CustomerId"].progress_apply(lambda x: cus_st[x][0])

100%|██████████| 332397/332397 [00:00<00:00, 983964.97it/s] 


In [23]:
cities = pd.read_excel("US_cities.xlsx")
state_city = {}
for st, ct in zip(cities["state"], cities["city"]):
    if st not in state_city:
        state_city[st] = [ct]
    else:
        state_city[st].append(ct)
        
        
state_mapper = {}
for st in df["Geography"]:
    if st not in state_mapper:
        state_mapper[st] = np.random.choice(list(state_city.keys()))
        
city_mapper = {}
for cust_id, st in zip(df["CustomerId"],df["Geography"]):
    if cust_id not in city_mapper:
        city_mapper[cust_id] = np.random.choice(state_city[state_mapper[st]])

In [24]:
df["City"] = df["CustomerId"].progress_apply(lambda x: city_mapper[x])
df["month"] = df["date"].progress_apply(lambda x: x.split("-")[1])
df["year"] = df["date"].progress_apply(lambda x: x.split("-")[-1])
df["CreditScore"] = df["CreditScore"].progress_apply(lambda x: round(x,0))
df["Account_type"] = df["Tenure"].progress_apply(lambda x: "Gold" if x > 15 else "Premium" if x <= 15 and x >=8 else "Normal")

100%|██████████| 332397/332397 [00:00<00:00, 1220946.22it/s]


In [25]:
df["Other Accounts"] = df["Tenure"].progress_apply(lambda x: 1 if x > 7 else 0)

100%|██████████| 332397/332397 [00:00<00:00, 1000911.81it/s]


In [26]:
df["Salary Account"] = df["Satisfaction Score"].progress_apply(lambda x: 1 if x >= 3 else 0 )

100%|██████████| 332397/332397 [00:00<00:00, 1069035.48it/s]


In [32]:
df.head(1)

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,...,Satisfaction Score,Card Type,Point Earned,date,City,month,year,Account_type,Other Accounts,Salary Account
0,15658241,Olmstead,141.0,Germany,Female,81,48,438091.980941,2,Yes,...,2,GOLD,2194.0,1-2-2023,Omaha,2,2023,Gold,1,0


In [30]:
df.columns

Index(['CustomerId', 'Surname', 'CreditScore', 'Geography', 'Gender', 'Age',
       'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'Exited', 'Complain', 'Satisfaction Score',
       'Card Type', 'Point Earned', 'date', 'City', 'month', 'year',
       'Account_type', 'Other Accounts', 'Salary Account'],
      dtype='object')

In [31]:
df["HasCrCard"] = df["HasCrCard"].progress_apply(lambda x: "Yes" if x == 1 else "No")
df["IsActiveMember"] = df["IsActiveMember"].progress_apply(lambda x: "Yes" if x == 1 else "No")

100%|██████████| 332397/332397 [00:00<00:00, 1272288.80it/s]


In [33]:
df["Exited"] = df["Exited"].progress_apply(lambda x: "Exited" if x == 1 else "Not Exited")

100%|██████████| 332397/332397 [00:00<00:00, 1272649.99it/s]


In [ ]:
Credit Score Category (Below 580: poor, 580 to 669: fair, 670 to 739: good, 740 to 799: very good, 800 and above: exceptional.) 


In [35]:
df["CreditScore"] = df["CreditScore"].progress_apply(lambda x: 350 if x <=350 else x)

100%|██████████| 332397/332397 [00:00<00:00, 1057899.54it/s]


In [41]:
def cs_category(x):
    if x > 800:
        return "Exceptional"
    if x > 740:
        return "Very Good"
    if x > 670:
        return "Good"
    if x > 580:
        return "Fair"
    else:
        return "poor"
df["CS_Category"] = df["CreditScore"].progress_apply(cs_category)

100%|██████████| 332397/332397 [00:00<00:00, 1012438.27it/s]


In [42]:
df["Satisfaction Score"].unique()

array([2, 1, 5, 4, 3])

In [43]:
df["SS_Category"] = df["Satisfaction Score"].progress_apply(lambda x: "Poor" if x <3 else "Good" if x < 4 else "Very Good")

100%|██████████| 332397/332397 [00:00<00:00, 1256563.98it/s]


In [ ]:
month_dic = {'May':5, 'Oct':10, 'Nov':11, 'Feb':2, 'Apr':4, 'Jul':7, 'Jan':1, 'Sep':9, 'Jun':6,
       'Aug':8, 'Dec':12, 'Mar':3}

def convert_month(x):
    temp_ = x.split("-")
    temp_[-2] = str(month_dic[temp_[-2]])
    return "-".join(temp_)

df["month"] = df["month"].progress_apply(convert_month)

In [44]:
df.to_csv("/data/bank_customer_churn_data.csv", index=False)